In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pylab as pl
import os
import pickle
import glob
import sklearn as skl
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler,LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut, LeaveOneGroupOut, GroupKFold
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import scipy as sp
import matplotlib.gridspec as gridspec
# import more_itertools as mit
from itertools import groupby
from operator import itemgetter
from tqdm import tqdm
import pandas as pd
import numpy as np
import pylab as pl
import scipy.io as sio
import seaborn as sns
import statsmodels.api as sm
from collections import Counter
import funcs as func
import matplotlib.pyplot as plt
import sys
import pdb
import glob
import pickle

import sys
sys.path.append('/home/jyotika/Utils/hddm/')

import hddm

data_dir = "./Data/processed_data/Y_maze/data_with_animal_ids/"
data_target_dir = "./Data/processed_data/Y_maze/data_with_animal_ids/for_b_cpp_calculation/"
data_target_dir2 = "./Data/processed_data/Y_maze/data_with_animal_ids/for_ddm_models/"
figure_dir = "./Figures/Y_maze/"

plt.rcParams["figure.facecolor"] = "w"
import sys


#from my_sklearn_tools.pca_regressors import LogisticPCR

sys.path.append("/home/bahuguna/Work/CBGT_CMU/analysis/")
from simulation_functions_loki import Simulation

postfix = "_with_binom"

In [ ]:
all_conflicts = pd.read_csv(data_dir+"all_experiments_df_with_DTs.csv")

In [ ]:
for grp in all_conflicts.groupby(["conflict","volatility","session"]):
    print(grp[1])


In [ ]:
block_changes = np.where(grp[1]["block"]!=grp[1]["block"].shift())[0]


In [ ]:
grp[1].iloc[:30]

In [ ]:
grp[0]

In [ ]:
ind_t0 = np.where(np.logical_and(grp[1]["block"]=="left",grp[1]["rewarded_code"]==1))[0]
bl_t0 = np.where(grp[1]["block"]=="left")[0]
ind_t1 = np.where(np.logical_and(grp[1]["block"]=="right",grp[1]["rewarded_code"]==1))[0]
bl_t1 = np.where(grp[1]["block"]=="right")[0]

In [ ]:
bl_t0

In [ ]:
unrewarded_t0 =  np.sort(list(set(bl_t0)-set(ind_t0)))

In [ ]:
unrewarded_t0

In [ ]:
ind_t1

In [ ]:
# Because Julia does not save her reward frames only the actions which were actually rewarded (which will be equivalent to the reward data frame only if performance == reward probability in the block)

def generate_reward_frame(rew,conf,n_trials):
    if conf == "No":
        ba,nba = 0.9,0.0
    elif conf == "Low":
        ba,nba = 0.9,0.1
    elif conf == "High":
        ba,nba = 0.75,0.25
    block_changes = np.where(rew["block"]!=rew["block"].shift())[0]
    ind_t0 = []
    ind_t1 = []
    for i,bc_ind in enumerate(block_changes):
        print(ba,nba)
        start = bc_ind
        if i < len(block_changes)-1:
            end = block_changes[i+1]
        else:
            end = -1
            
        dat_slice = rew.iloc[start:end].copy()
        block = np.unique(dat_slice["block"])
        if len(block)==0:
            continue
        #print("block",block)
        if len(block)>1:
            print("problem !!")
        else:
            block = block[0]
        print(block)

        if block == "left":
            # Check which trials are rewarded
            ind_rewarded_opt = start+np.where(np.logical_and(dat_slice["rewarded_code"]==1,dat_slice["chosen_action"]=="left"))[0]
            ind_rewarded_subopt = start+np.where(np.logical_and(dat_slice["rewarded_code"]==1,dat_slice["chosen_action"]=="right"))[0]
            all_opt = start+np.where(dat_slice["chosen_action"]=="left")[0]
            all_subopt = start+np.where(dat_slice["chosen_action"]=="right")[0]
        elif block == "right":
            ind_rewarded_subopt = start+np.where(np.logical_and(dat_slice["rewarded_code"]==1,dat_slice["chosen_action"]=="left"))[0]
            ind_rewarded_opt = start+np.where(np.logical_and(dat_slice["rewarded_code"]==1,dat_slice["chosen_action"]=="right"))[0]
            all_opt = start+np.where(dat_slice["chosen_action"]=="right")[0]
            all_subopt = start+np.where(dat_slice["chosen_action"]=="left")[0]

        print("rewarded_opt",ind_rewarded_opt,len(ind_rewarded_opt))
        print("rewarded_subopt",ind_rewarded_subopt,len(ind_rewarded_subopt))


        num_b,num_un = int(len(dat_slice)*ba),int(len(dat_slice)*nba)
        print("should_be_rewarded_opt",num_b)
        print("should_be_rewarded_subopt",num_un)
#         if block == "left":
#             print("left,right =",num_b,num_un )
#         elif block == "right":
#             print("right,left =",num_b,num_un )

#         ind_unrewarded_opt = list(set(np.arange(start,end,1))-set(ind_rewarded_opt)) #- set(ind_rewarded_t1))
#         ind_unrewarded_subopt = list(set(np.arange(start,end,1))-set(ind_rewarded_subopt)) #- set(ind_rewarded_t0))    
        ind_unrewarded_opt = list(set(all_opt)-set(ind_rewarded_opt)) #- set(ind_rewarded_t1))
        ind_unrewarded_subopt = list(set(all_subopt)-set(ind_rewarded_subopt)) #- set(ind_rewarded_t0))    
        rem_pool = list(set(np.arange(start,end,1)) - set(ind_rewarded_opt) - set(ind_unrewarded_opt) - set(ind_rewarded_subopt) - set(ind_unrewarded_subopt))
        pool_for_opt = ind_unrewarded_subopt #list(set(np.arange(start,end,1))- set(ind_rewarded_opt) - set(ind_rewarded_subopt))
        pool_for_subopt = ind_unrewarded_opt
        
        print("rem_pool",rem_pool)
        
        print("unrewarded_opt",ind_unrewarded_opt)
        print("unrewarded_subopt",ind_unrewarded_subopt)
        print("pool",pool_for_opt)
        num_to_be_rewarded_b = num_b - len(ind_rewarded_opt)
        num_to_be_rewarded_nb = num_un - len(ind_rewarded_subopt)

        print(num_to_be_rewarded_b,num_to_be_rewarded_nb)
        
        rem_b = 0
        if num_to_be_rewarded_b > 0:
            if num_to_be_rewarded_b> len(pool_for_opt):
                rem_b = num_to_be_rewarded_b - len(pool_for_opt)
                num_to_be_rewarded_b = len(pool_for_opt)
#                 if rem_b > len(rem_pool):
#                     rem_b = len(rem_pool)
            else:
                rem_b = 0
                
            to_be_rewarded_b = list(np.random.choice(pool_for_opt,num_to_be_rewarded_b,replace=False)) #+ list(np.random.choice(rem_pool,rem_b,replace=False))
                                                         
        else:
            to_be_rewarded_b = []
         
        rem_nb = 0
        if num_to_be_rewarded_nb > 0:
#             rem_nb = ind_unrewarded_opt   #list(set(ind_unrewarded_subopt) -set(to_be_rewarded_b))
            if num_to_be_rewarded_nb >len(pool_for_subopt):
                rem_nb = num_to_be_rewarded_nb - len(pool_for_subopt)                                                                                              
                num_to_be_rewarded_nb =  len(pool_for_subopt)#num_to_be_rewarded_nb
#                 if rem_nb > len(rem_pool):
#                     rem_nb = len(rem_pool)
#             else:
            else:
                rem_nb = 0
#                 num_to_be_rewarded_nb = len(rem_nb)
            to_be_rewarded_nb = list(np.random.choice(pool_for_subopt,num_to_be_rewarded_nb,replace=False)) #+ list(np.random.choice(rem_pool,rem_nb,replace=False))
                
        else:
            to_be_rewarded_nb = []
        
        # where to put rem_pool
        if rem_b > rem_nb:
            to_be_rewarded_b = np.hstack((to_be_rewarded_b,rem_pool))
        else:
            to_be_rewarded_nb = np.hstack((to_be_rewarded_nb,rem_pool))
        # add whatever remains to opt
#         rem = list(set(rem_nb)-set(to_be_rewarded_nb))
            
        final_ind_rewarded_opt = np.sort(np.hstack((ind_rewarded_opt,to_be_rewarded_b)))
        final_ind_rewarded_subopt = np.sort(np.hstack((ind_rewarded_subopt,to_be_rewarded_nb)))
        print("opt_before",ind_rewarded_opt)
        print("subopt_before",ind_rewarded_subopt)

        print("opt_after",final_ind_rewarded_opt)
        print("subopt_after",final_ind_rewarded_subopt)
        reward_prob = (len(final_ind_rewarded_opt)+len(final_ind_rewarded_subopt))/len(dat_slice)
        print("checkin rew prob",ba,nba,reward_prob)
        if ((ba+nba)-reward_prob) > 0.1:
            print("Problems !!!")
            print(dat_slice)
            #pdb.set_trace()
        
        print("intersection",set(final_ind_rewarded_opt).intersection(final_ind_rewarded_subopt))
        if len(set(final_ind_rewarded_opt).intersection(final_ind_rewarded_subopt)) != 0:
            print("same action rewarded twice")
            pdb.set_trace()
        
        if block == "left":
            ind_t0.append(final_ind_rewarded_opt)
            ind_t1.append(final_ind_rewarded_subopt)
        elif block == "right":
            ind_t0.append(final_ind_rewarded_subopt)
            ind_t1.append(final_ind_rewarded_opt)
            
        #print(rew.iloc[final_ind_rewarded_subopt][["chosen_action","block","rewarded"]])
#     print(ind_t0,ind_t1)
    
    return np.hstack(ind_t0).astype(int), np.hstack(ind_t1).astype(int)





def conv_rew_df_t_epochs_fmt(rew,cond,conf,vol,session):
    # Trial is constant, and first action is always left
    # For the general case, save the t_epochs df during the simulation
    # t0 == left
    #r_df = rew.loc[rew["data_type"]=="reward_df"]
    n_trials = len(rew)
    reward = np.random.normal(loc=1.0,scale=0.01,size=n_trials)
    #print(reward)
    t_epochs = pd.DataFrame(columns=["r_t0","r_t1","cp","epoch_number","reward_p_t0","session","conflict","volatility","condition","p_id_solution","action_history","chosen_action","trial_num","animal_id"])
    n_trials =  len(rew) #int(np.unique(rew["n_trials"])[0])
    cp = np.zeros(n_trials)
    #print(n_trials)
    cp_idx = np.where(rew["block"]!=rew["block"].shift())
    cp[cp_idx] = 1
    
    r_t0 = np.zeros(n_trials) # always left
    r_t1 =np.zeros(n_trials)
    
    ind_t0,ind_t1 = generate_reward_frame(rew,conf,n_trials)
#     print("ind_t0",ind_t0)
#     print("ind_t1",ind_t1)
    r_t0[ind_t0] = reward[ind_t0]
    r_t1[ind_t1] = reward[ind_t1]
    t_epochs["r_t0"] = r_t0
    t_epochs["r_t1"] = r_t1
    
    all_inds = np.arange(0,n_trials)
#     print("all_inds",all_inds)
    #print("t_epochs",t_epochs)
    t_epochs["cp"] = cp
    t_epochs["epoch_number"] = rew.iloc[all_inds]["block_num"].values
    block = list(rew.iloc[all_inds]["block"])
    chosen_action = list(rew.iloc[all_inds]["chosen_action"])
    
    rew_p_t0 = np.zeros(n_trials)
    rew_p_t0[ind_t0] = 1.0
    t_epochs["reward_p_t0"] = rew_p_t0
    t_epochs["chosen_action"] = chosen_action
    t_epochs["rewarded"] = rew["rewarded"].values
    t_epochs["action_history"] = [ 0 if x == "left" else 1 for x in chosen_action]
    t_epochs["p_id_solution"] = list(block)
    t_epochs["optimal"] = list(block)
    t_epochs["conflict"] = conf
    t_epochs["condition"] = cond
    t_epochs["volatility"] = vol
    t_epochs["session"] = session
    t_epochs["RT(ms)"] = rew.iloc[all_inds]["RT(ms)"].values
    t_epochs["animal_id"] = rew.iloc[all_inds]["animal_id"].values
    t_epochs["trial_num"] = list(rew.iloc[all_inds]["trial_num"])
    t_epochs["block_num"] = list(rew.iloc[all_inds]["block_num"])
    
    #print("final")
    print("====================================")
    print(t_epochs.loc[(t_epochs["chosen_action"]!=t_epochs["optimal"])&(t_epochs["rewarded"]=="rewarded")][["optimal","chosen_action","r_t0","r_t1","rewarded"]])
    
    
    return t_epochs
    #print(t_epochs)
    
    

In [ ]:
# Separate the batches into individual trials and save them as seperate .csv
for grp in all_conflicts.groupby(["conflict","volatility","condition","session"]): # Usually session is a proxy for animal id
#     if seed == '2807188':
#         continue
    rew = grp[1].copy()
    if len(np.unique(rew["animal_id"]))>1:
        print("more than one animal in this session !!")
    #print(grp[0])
    #print(rew.loc[(rew["chosen_action"]!=rew["block"])&(rew["rewarded"]=="rewarded")][["block","chosen_action","rewarded"]])
    cond = grp[0][2]
    conf = grp[0][0]
    vol = grp[0][1]
    sess = grp[0][3]
    print("actual_data")
    print("==========================================")
    print(rew.loc[(rew["chosen_action"]!=rew["block"])&(rew["rewarded"]=="rewarded")][["block","chosen_action","rewarded"]])
    print(len(rew.loc[(rew["chosen_action"]!=rew["block"])&(rew["rewarded"]=="rewarded")][["block","chosen_action","rewarded"]]))
    
    
    b_cpp_compatible_format = conv_rew_df_t_epochs_fmt(rew,cond,conf,vol,sess)
    b_cpp_compatible_format = b_cpp_compatible_format.rename(columns={'RT(ms)':'rt'})
    b_cpp_compatible_format["rewarded"] = [1 if x =="rewarded" else 0  for x in rew["rewarded"]]
    print("animal_id",np.unique(b_cpp_compatible_format["animal_id"]))
    print("================================")
    print(b_cpp_compatible_format.loc[(b_cpp_compatible_format["chosen_action"]!=b_cpp_compatible_format["optimal"])&(b_cpp_compatible_format["rewarded"]==1)][["optimal","chosen_action","r_t0","r_t1","rewarded"]])
    print(len(b_cpp_compatible_format.loc[(b_cpp_compatible_format["chosen_action"]!=b_cpp_compatible_format["optimal"])&(b_cpp_compatible_format["rewarded"]==1)][["optimal","chosen_action","r_t0","r_t1","rewarded"]]))
    print(cond,conf,vol,sess)
    filename = cond+"_"+conf+"_"+vol+"_"+str(sess)+".csv"
    b_cpp_compatible_format.to_csv(data_target_dir+filename)

In [ ]:
rew.loc[(rew["chosen_action"]!=rew["block"])&(rew["rewarded"]=="rewarded")][["block","chosen_action","rewarded"]]

In [ ]:
b_cpp_compatible_format.loc[(b_cpp_compatible_format["chosen_action"]!=b_cpp_compatible_format["optimal"])&(b_cpp_compatible_format["rewarded"]==1)][["r_t0","r_t1","optimal","chosen_action","rewarded"]]

In [ ]:
b_cpp_compatible_format[["chosen_action","optimal","rewarded"]]

In [ ]:
rew[["chosen_action","block","rewarded"]]

In [ ]:
b_cpp_compatible_format


In [ ]:
# subj_data_files = glob.glob(data_target_dir+'*[!block]_[0-9]*[!with_b_cpp].csv')

In [ ]:
subj_data_files = glob.glob(data_target_dir+'*[!block]_*[0-9]*.csv')

In [ ]:
subj_data_files

In [ ]:
for x in subj_data_files:
    if "iSPN-Control_High_Low_0" in x:
        print("found!!")

In [ ]:
filen = subj_data_files[0]
filen.split('/')[-1].split('_')[3].split('.csv')[0]

In [ ]:
# import os
# os.chdir("/home/bahuguna/Work/CBGT_CMU/analysis/")

#data_target_dir1 = "../cbgt2_plasticity/Data/var_lambda/10/for_b_cpp_calculation/"
# data_target_dir1 = "../cbgt2_plasticity/Data/competition/for_b_cpp_calculation/"
print(os.getcwd())
# cmd = "python binary_ddm_rl_simulation_p.py ymaze "
cmd = "python binary_ddm_rl_simulation_p.py ymaze_binom "
cmd = cmd + data_target_dir
output = os.system(cmd)




In [ ]:
print(output)

In [ ]:
#b_cpp_path = '/home/bahuguna/Work/CBGT_CMU/cbgt2_plasticity/Data/competition/for_b_cpp_calculation/simulated_data/'
# b_cpp_path = data_target_dir+'/simulated_data/'
b_cpp_path = data_target_dir+'/simulated_data_binom/'

files = glob.glob(b_cpp_path+"*[0-9]*.pkl")
import pickle

print(files)

final_b_cpp_df = pd.DataFrame()

for f in files:
    print(f)
    if "with" in f.split('/')[-1]:
        continue
    print(f)
    cond = f.split('/')[-1].split('_condition_')[1].split('_')[0]
    conf = f.split('/')[-1].split('_conflict_')[1].split('_')[0]
    session = f.split('_session_')[1].split('.pkl')[0]
    volatility = f.split('_volatility_')[1].split('_')[0]
    
    
    filesrc = cond+"_"+conf+"_"+volatility+"_"+session+".csv"
    filedest = cond+"_"+conf+"_"+volatility+"_"+session+"with_b_cpp.csv"
    
    temp = pd.read_csv(data_target_dir+filesrc)
    #print(temp["rt"])
    sim = pickle.load(open(f,"rb"))
    ideal_B = sim.ideal_B
    cpp = sim.CPP
    b_t0 = sim.B[:,0]
    b_t1 = sim.B[:,1]
    MC = sim.MC
    lr = sim.lr
    hr = sim.H
    sF = sim.sF
    u_val = sim.u_val_all
    n_val = sim.n_val_all
    
    
    temp["cpp"] = cpp
    temp["ideal_B"] = ideal_B
    temp["b_t0"] = b_t0 
    temp["b_t1"] = b_t1
    temp["MC"] = MC
    temp["learning_rate"] = lr
    temp["H(hazard_rate)"] = hr
    temp["sigma_estimated"] = sF
    temp["rpe_t0"] = sim.rpe[:,0]
    temp["rpe_t1"] = sim.rpe[:,1]    
    temp["u_val"] = u_val
    temp["n_val"] = n_val
    print("animal_id",np.unique(temp["animal_id"]))
    temp.to_csv(data_target_dir+filedest)
    final_b_cpp_df = final_b_cpp_df.append(temp)
    
    
    #print(temp)
    
final_b_cpp_df.to_csv(data_target_dir+"for_av_fits_hddm_test.csv")


In [ ]:
sim.rpe[:,1]

In [ ]:
final_b_cpp_df.columns

In [ ]:
# final_b_cpp_df = pd.read_csv(data_target_dir+"for_av_fits_hddm.csv")

In [ ]:
np.unique(final_b_cpp_df["animal_id"])

In [ ]:
final_b_cpp_df.loc[(final_b_cpp_df["chosen_action"]!=final_b_cpp_df["optimal"])&(final_b_cpp_df["rewarded"]==1)][["chosen_action","optimal","rewarded","r_t0","r_t1"]]

In [ ]:
final_b_cpp_df.loc[final_b_cpp_df["condition"]=="iSPN-Stim"].iloc[:100][["optimal","chosen_action","r_t0","r_t1","rewarded","ideal_B","condition","conflict","volatility","cp","b_t0","b_t1"]]

In [ ]:
test = final_b_cpp_df.loc[final_b_cpp_df["condition"]=="iSPN-Stim"].iloc[:40].copy()

In [ ]:
np.unique(test["volatility"])

In [ ]:
test["session + block_num + trial_num"] = [str(x)+"-"+str(z)+"-"+str(y)  for x,y,z in zip(test["session"],test["trial_num"],test["block_num"])]

In [ ]:
test[["r_t0","r_t1"]].mean()

In [ ]:
test[["r_t0","r_t1","b_t0","b_t1","chosen_action","optimal","ideal_B","cpp","learning_rate","rewarded"]]

In [ ]:
fig,ax = pl.subplots(1,1,figsize=(15,6))
sns.lineplot(x="session + block_num + trial_num",y="b_t0",data=test,ax=ax,label='b_t0',marker='o')
sns.lineplot(x="session + block_num + trial_num",y="b_t1",data=test,ax=ax,label='b_t1',marker='o')
sns.lineplot(x="session + block_num + trial_num",y="r_t1",data=test,ax=ax,label='r_t1',marker='o')
sns.lineplot(x="session + block_num + trial_num",y="r_t0",data=test,ax=ax,label='r_t0',marker='o')
sns.lineplot(x="session + block_num + trial_num",y="cp",data=test,ax=ax,label='cp',marker='o')

In [ ]:
sns.catplot(x="conflict",y="H(hazard_rate)",col="volatility",data=final_b_cpp_df,kind='point',hue='condition')

In [ ]:
for grp in final_b_cpp_df.groupby(["condition","conflict","volatility","session"]):
    print(grp[0])
    print(len(np.where(grp[1]["cp"]==1)[0]), len(grp[1]))

In [ ]:
sns.catplot(x='conflict',y="ideal_B",col='volatility',hue='condition',data=final_b_cpp_df,kind='bar')

In [ ]:
(0.07*0.5)/(0.5*0.07+0.75*0.93)
        

In [ ]:
(0.07*0.5)/(0.5*0.07+0.9*0.93)
